In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ttest_rel
from statsmodels.stats.power import TTestIndPower

#### Исследовательские вопросы. Какие метрики просели, а какие выросли? Что и когда можно обобщать на генеральную совокупность.

#### Зачем исследование. Хотим принести компании больше денег. Достигнуть этого попробуем через передвижение ограничительных врат с 30го на 40й уровень. Чтобы случайно не потерять деньги делаем а/б тест.

#### Данные и методология ниже.

# Data

In [ ]:
df = pd.read_csv("cookie_cats.csv")
df = df.rename(columns={'userid':'id', 'version':'group', 'sum_gamerounds':'hingame', 
                        'retention_1':'cb1', 'retention_7':'cb7'})
df.group[df.group=='gate_30'] = 'a'
df.group[df.group=='gate_40'] = 'b'
df.cb1[df.cb1==True]=1
df.cb1[df.cb1==False]=0
df.cb7[df.cb7==True]=1
df.cb7[df.cb7==False]=0
df = df.drop('id', axis=1)
df = df.drop(57702) #сломаное значение в датасете

#### Все записи уникальные поэтому никаких сумм по id делать не будем, чтобы улучшить данные

In [ ]:
print(len(df[(df['group']=='a')]), len(df[(df['group']=='b')]))
print(len(df[(df['group']=='a')&(df['cb1']==1)]), len(df[(df['group']=='b')&(df['cb1']==1)]))
print(len(df[(df['group']=='a')&(df['cb7']==1)]), len(df[(df['group']=='b')&(df['cb7']==1)]))

In [ ]:
lst = [df[(df['group']=='a')].hingame.mean(), df[(df['group']=='b')][:44699].hingame.mean(),
       df[(df['group']=='a')].cb1.mean(), df[(df['group']=='b')][:20034].cb1.mean(),
       df[(df['group']=='a')][:8279].cb7.mean(), df[(df['group']=='b')].cb7.mean(),
       df[(df['group']=='a')&(df['cb1']==1)].hingame.mean(), df[(df['group']=='b')&(df['cb1']==1)][:20034].hingame.mean(),
       df[(df['group']=='a')&(df['cb7']==1)][:8279].hingame.mean(), df[(df['group']=='b')&(df['cb7']==1)].hingame.mean()]
print(lst)

In [ ]:
alpha = 0.05
st = ttest_rel(df[(df['group']=='a')].hingame, df[(df['group']=='b')][:44699].hingame)
print('Metric diffs is {}significant\n'.format( {True:'', False:'not '}[st[1] < alpha]), end='')
print(st, end='\n\n')
st = ttest_rel(df[(df['group']=='a')].cb1, 
               df[(df['group']=='b')][:44699].cb1)
print('Metric diffs is {}significant\n'.format( {True:'', False:'not '}[st[1] < alpha]), end='')
print(st, end='\n\n')
st = ttest_rel(df[(df['group']=='a')].cb7, 
               df[(df['group']=='b')][:44699].cb7)
print('Metric diffs is {}significant\n'.format( {True:'', False:'not '}[st[1] < alpha]), end='')
print(st, end='\n\n')
st = ttest_rel(df[(df['group']=='a')&(df['cb1']==1)].hingame, 
               df[(df['group']=='b')&(df['cb1']==1)][:20034].hingame)
print('Metric diffs is {}significant\n'.format( {True:'', False:'not '}[st[1] < alpha]), end='')
print(st, end='\n\n')
st = ttest_rel(df[(df['group']=='a')&(df['cb7']==1)][:8279].hingame, 
               df[(df['group']=='b')&(df['cb7']==1)].hingame)
print('Metric diffs is {}significant\n'.format( {True:'', False:'not '}[st[1] < alpha]), end='')
print(st, end='\n\n')

##### 1. Разницы между группами а и б по количеству наигранных часов - нет.
##### 2. Есть различия при уровне альфа выше 10.6%, так как уровень очень высок, то считаем, что изменений нет в количестве пользователей возвращающихся через 1 день.
##### 3. На всех стандартных уровнях значимости $\alpha \in \{0.01, 0.05, 0.1\}$есть различия в количестве пользователей возвращающихся через 7 дней.
##### 4. Различия в количестве раундов в игре, для тех кто возвращается через 1 день, нет на всех уровнях значимости.
##### 5. Есть различия в количестве раундов в игре, для тех кто возвращается через 7 дней, на уровне значимости $\alpha = 0.1$

# Group size for test

In [ ]:
(df[(df['group']=='a')].cb7.mean()-df[(df['group']=='b')][:44699].cb7.mean())/df[(df['group']=='a')].cb7.mean()

In [ ]:
series_1 = df[(df['group']=='a')].cb7
series_2 = df[(df['group']=='b')][:44699].cb7
power=0.8
alpha=0.05
effect = 0.04081872720856373
analysis = TTestIndPower()
result = analysis.solve_power(effect, power=power, nobs1=None, ratio=1.0, 
                              alpha=alpha)
print('Sample Size for metric: %.3f' % result)
print(f'Now sample size is {len(series_1)}\n')

In [ ]:
(df[(df['group']=='a')&(df['cb7']==1)][:8279].hingame.mean()
 -df[(df['group']=='b')&(df['cb7']==1)].hingame.mean())/df[(df['group']=='a')&(df['cb7']==1)][:8279].hingame.mean()

In [ ]:
series_1 = df[(df['group']=='a')&(df['cb7']==1)][:8279].hingame
series_2 = df[(df['group']=='b')&(df['cb7']==1)].hingame
power=0.8
alpha=0.05
effect = 0.032792342027199856
analysis = TTestIndPower()
result = analysis.solve_power(effect, power=power, nobs1=None, ratio=1.0, 
                              alpha=alpha)
print('Sample Size for metric: %.3f' % result)
print(f'Now sample size is {len(series_1)}\n')

1. Можно раскатывать на всю генеральную совокупность информацию о том что в группе Б по сравнению с группой А на 4% упало количество пользователей возвращающихся через 7 дней. Так как сейчас размер каждой выборки 44699, а надо чтобы было хотябы 9423.
2. Чтобы говорить, что в генеральной совокупности среднее количество раундов через семь дней выросло в группе Б на 3.28% по сравнению с группой А нужно подождать пока в каждой группе наберется 14600 наблюдений, сейчас есть 8279. Так как эти данные собраны за 14 дней, то нужно подождать еще $\dfrac{14*14600}{8279} - 14 = 24.7 - 14 = 10.7$ дней.


## Вывод
Менеджеру говорим, что изменения есть, увеличился отток пользователей через 7 дней на 4%. Если хочет знать насколько выросло количество сыгранных раундов, то пусть подойдет через 12 дней, сейчас есть рост на 3.28% в группе Б по сравнению с группой А.


Вводить ли изменения в случае подтверждения цифр выше под вопросом, нужно посмотреть как изменилось количество просмотренной рекламы. Сейчас можно сказать, что упало количество пользователей через 7 дней и выросла нагрузка на серверы, так как те кто остался стали чаще играть.

In [ ]:
plt.figure(figsize=(7, 5), dpi=80)
plt.hist(df[(df['group']=='a')].cb7, label='A', alpha=0.5, bins=2)
plt.hist(df[(df['group']=='b')][:44699].cb7, label='B', alpha=0.5, bins=2)
plt.grid()
plt.legend(loc=1, prop={'size': 15})
plt.ylabel("Количество пользователей")
plt.xlabel("0-ушли, 1-остались")
plt.title("Распределение возвращения пользователей через 7 дней после установки")
plt.savefig("cb7.png", facecolor = 'white')
plt.show()

In [ ]:
plt.figure(figsize=(7, 5), dpi=80)
plt.hist(df[(df['group']=='a')].cb1, label='A', alpha=0.5, bins=2)
plt.hist(df[(df['group']=='b')][:44699].cb1, label='B', alpha=0.5, bins=2)
plt.grid()
plt.legend(loc=1, prop={'size': 15})
plt.ylabel("Количество пользователей")
plt.xlabel("0-ушли, 1-остались")
plt.title("Распределение возвращения пользователей через 1 день после установки")
plt.savefig("cb1.png", facecolor = 'white')
plt.show()

In [ ]:
plt.figure(figsize=(7, 5), dpi=80)
a = df[(df['group']=='a')].hingame.sort_values().values
b = df[(df['group']=='b')][:44699].hingame.sort_values().values
plt.hist(a, label='A', alpha=0.5, bins=25)
plt.hist(b, label='B', alpha=0.5, bins=25)
plt.grid()
plt.legend(loc=1, prop={'size': 15})
plt.ylabel("Количество пользователей")
plt.xlabel("Количество раундов")
plt.title("Раундов в игре")
plt.savefig("hingame.png", facecolor = 'white')
plt.show()

In [ ]:
plt.figure(figsize=(7, 5), dpi=80)
a = df[(df['group']=='a')&(df['cb1']==1)].hingame, 
b = df[(df['group']=='b')&(df['cb1']==1)][:20034].hingame
plt.hist(a, label='A', alpha=0.5, bins=25)
plt.hist(b, label='B', alpha=0.5, bins=25)
plt.grid()
plt.legend(loc=1, prop={'size': 15})
plt.ylabel("Количество пользователей")
plt.xlabel("Количество раундов")
plt.title("Раундов в игре тех, кто вернулся через день")
plt.savefig("hingame1.png", facecolor = 'white')
plt.show()

In [ ]:
plt.figure(figsize=(7, 5), dpi=80)
a = df[(df['group']=='a')&(df['cb7']==1)][:8279].hingame
b = df[(df['group']=='b')&(df['cb7']==1)].hingame
plt.hist(a, label='A', alpha=0.5, bins=25)
plt.hist(b, label='B', alpha=0.5, bins=25)
plt.grid()
plt.legend(loc=1, prop={'size': 15})
plt.ylabel("Количество пользователей")
plt.xlabel("Количество раундов")
plt.title("Раундов в игре тех, кто вернулся через 7 дней")
plt.savefig("hingame7.png", facecolor = 'white')
plt.show()